# Generate Market Book by Order (MBO) quotes using dedicated service - Python

### Overview
This sample is designed to request quotes and illustrate full order book messaging.

The Market Book by Order (MBO) describes an order-based (or quote-based) data feed that provides the ability to view individual queue position, the full order book (all the depths) and the details of each individual order or quote at each price level.

**Note : This sample is not about Market Book by Limit (MBL). For the MBL please refer the notebook [Tick] bbo10_and_trades**

### Inputs/outputs
Quotes extraction sample requires instrument's identifier, date time intervals as per inputs and returns the occured quotes information.

### Services used
This sample uses *gRPC requests* in order to retrieve quotes information from the hosted service. The queried endpoint in this script are:
* *TickQuotesService*: to directly retrieve quotes data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    
***

# Run MBO Quotes sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly

In [ ]:
pip install systemathics.apis==2.36.* --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_quotes_pb2 as tick_quotes
import systemathics.apis.services.tick.v1.tick_quotes_pb2_grpc as tick_quotes_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve data
To request *tick trades* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
# Tha data is provided by ICE : let's use the ICE mapping codes to generate the identifier
# The ICE ticker
ticker = 'E:BNP'
# The source for full order book : Euronext
source = 'EQUITY_L2_973'

#### 3.2 Time period delimitation

In [ ]:
# Create time intervals (we are using Google date format)
# Full order book data avaialble (sample) : from 2021-11-01 to 2021-11-12
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 11, day = 1), 
    end_date = date.Date(year = 2021, month = 11, day = 1)
)

# Build the tick quotes request time interval (we are using Google date time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 12, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 12, minutes = 30, seconds = 0)
)

In [ ]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

#### 3.3 Request creation

In [ ]:
# generate the tick trades request
my_identifier = identifier.Identifier(exchange = source, ticker = ticker)
request = tick_quotes.TickQuotesRequest(
    identifiers = [my_identifier],
    constraints = my_constraints
)

#### 3.4 Request processing and result export


In the following code snippet, we request quotes for the given instrument.
The streamed response is exported on the fly in a file.

In [ ]:
# small method to convert the side
def prettyprint(side):
    if side == 1:
        return "Bid"
    else:
        return 'Ask'

In [ ]:
import csv
from datetime import timedelta

os.makedirs('output', exist_ok=True)
filename = 'output/{0}_quotes.csv'.format(ticker)
quotes_count = 0
quotes_count_intern = 0

with open(filename, mode='w') as quotes_file:
    quotes_writer = csv.writer(quotes_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # write 1rst row
    quotes_writer.writerow(['Timestamp', 'ID', 'Side', 'Price', 'Size', 'Condition'])
    try:
        # open a gRPC channel
        with channel_helpers.get_grpc_channel() as channel:  

            # instantiate the tick quotes service
            service = tick_quotes_service.TickQuotesServiceStub(channel)

            # process the tick quotes request
            metadata = [('authorization', token)]
            for current in service.TickQuotes(request=request, metadata=metadata):
                
                # filter 1rst line = mapping (not used here since we are using only one instrument in request)
                if current.data.time_stamp.seconds > 0:
                    quotes_count +=1
                    
                    time =datetime.fromtimestamp(current.data.time_stamp.seconds) + timedelta(microseconds = current.data.time_stamp.nanos/1000)
                    for quote in current.data.quotes:
                        quotes_count_intern +=1
                        my_id = quote.id
                        my_side = quote.side
                        my_price= quote.price.value
                        my_size= quote.size.value
                        my_condition= quote.condition.value
                        quotes_writer.writerow([time, my_id, prettyprint(my_side), my_price, my_size, my_condition])

    except grpc.RpcError as e:
        display(e.code().name)
        display(e.details())

The following code snippet displays the total retrieved quotes count:

In [ ]:
print('Total quotes retrieved: {0}'.format(quotes_count))